In [1]:
! pip install --quiet --upgrade nltk gensim > /dev/null

In [2]:
# importing the libraries
import pandas as pd
import pickle
import re
import string
import numpy as np
import gensim
from gensim import corpora
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
from pprint import pprint
import warnings
warnings.filterwarnings("ignore")

In [3]:
import os

# Identify the working directory and data files
working_directory = "40-LDA-42-Topics-5-Words"

# Create the working directory if needed
try:
    os.makedirs(working_directory, exist_ok=True)
except OSError as error:
    print(f"Error creating {working_directory}: {error}")

In [4]:
selected_num_topics = 42
num_words = 5

In [5]:
lda_model = pd.read_pickle("./35-LDA_and_NMF_finalized/lda_model_42_topics.pickle")

In [6]:
# Extract the top 5 words for each topic
top_words = lda_model.print_topics(num_topics=selected_num_topics, num_words=num_words)

In [7]:
top_words

[(0,
  '0.760*"report" + 0.041*"adverse" + 0.041*"submit" + 0.040*"impact" + 0.035*"upon"'),
 (1,
  '0.398*"injury" + 0.321*"intervention" + 0.279*"medical" + 0.000*"report" + 0.000*"doc"'),
 (2,
  '0.417*"confirmation" + 0.158*"could" + 0.112*"allegation" + 0.096*"determine" + 0.096*"probable"'),
 (3,
  '0.358*"fail" + 0.328*"error" + 0.313*"transmitter" + 0.000*"occur" + 0.000*"report"'),
 (4,
  '0.333*"could" + 0.287*"probable" + 0.213*"determine" + 0.165*"cause" + 0.000*"allegation"'),
 (5,
  '0.306*"could" + 0.289*"determine" + 0.259*"cause" + 0.142*"root" + 0.001*"possible"'),
 (6,
  '0.994*"report" + 0.001*"occur" + 0.000*"transmitter" + 0.000*"allegation" + 0.000*"intervention"'),
 (7,
  '0.146*"issue" + 0.114*"resolve" + 0.095*"replace" + 0.066*"support" + 0.060*"close"'),
 (8,
  '0.163*"information" + 0.162*"available" + 0.105*"patient" + 0.089*"additional" + 0.088*"supplemental"'),
 (9,
  '0.269*"hour" + 0.263*"one" + 0.248*"signal" + 0.208*"loss" + 0.007*"occur"'),
 (10,
  

In [8]:
topics_with_scores_df = pd.DataFrame()

for idx, topic in lda_model.show_topics(num_topics=selected_num_topics, num_words=num_words, formatted=False):
    top_words = [(word, round(prob, 4)) for word, prob in topic]    
    row_dict = {'Topic': str(int(idx+1))}
    for i, (word, prob) in enumerate(top_words):
        row_dict[f"Word {i+1}"] = word
        row_dict[f"Weight {i+1}"] = str(round(prob, 4))
    topics_with_scores_df = topics_with_scores_df.append(row_dict, ignore_index=True)

with pd.option_context('display.max_colwidth', None):
    display(topics_with_scores_df)

,Topic,Word 1,Weight 1,Word 2,Weight 2,Word 3,Weight 3,Word 4,Weight 4,Word 5,Weight 5
0,1,report,0.7596,adverse,0.0413,submit,0.0409,impact,0.0398,upon,0.0349
1,2,injury,0.398,intervention,0.3205,medical,0.279,report,0.0,doc,0.0
2,3,confirmation,0.4171,could,0.1582,allegation,0.1121,determine,0.0965,probable,0.0957
3,4,fail,0.3576,error,0.3275,transmitter,0.313,occur,0.0004,report,0.0
4,5,could,0.3327,probable,0.2869,determine,0.2133,cause,0.165,allegation,0.0
5,6,could,0.3064,determine,0.2886,cause,0.259,root,0.1422,possible,0.0011
6,7,report,0.9937,occur,0.0009,transmitter,0.0,allegation,0.0,intervention,0.0
7,8,issue,0.1456,resolve,0.1142,replace,0.0945,support,0.0657,close,0.0604
8,9,information,0.163,available,0.1616,patient,0.1049,additional,0.089,supplemental,0.0879
9,10,hour,0.269,one,0.2631,signal,0.2483,loss,0.2084,occur,0.0072


In [9]:
topics_with_scores_df.to_pickle(f"{working_directory}/lda_topics_with_scores_df_{selected_num_topics}_topics_5_words.pickle")
topics_with_scores_df.to_csv(f"{working_directory}/lda_topics_with_scores_df_{selected_num_topics}_topics_5_words.csv", index=False)